
# 🌌 Hipparcos Stellar Clustering — Starter Notebook

This notebook helps you cluster **real stars** from the **Hipparcos** mission using features like **color index (B–V)**, **absolute magnitude (Mv)**, and **distance** (from parallax).
It’s designed to be clean, reproducible, and portfolio-ready.

**What you’ll do:**
1. Load Hipparcos data (CSV or FITS converted to CSV).
2. Engineer features: distance (pc), absolute magnitude, color index.
3. Explore with an HR diagram (B–V vs Mv).
4. Cluster stars (KMeans; optional DBSCAN) and visualize results.
5. Profile clusters and save outputs.

> Tip: Start with **~10k–50k rows** for speed. You can scale up later.



## 📦 Setup

Run the next cell once to install packages if needed (comment it out if already installed).


In [ ]:

# !pip install pandas numpy scikit-learn matplotlib plotly seaborn
# Optional if you plan to query ESA archives directly:
# !pip install astroquery


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN
from sklearn.metrics import silhouette_score

# Nice plotting defaults
plt.rcParams["figure.figsize"] = (7,5)
plt.rcParams["axes.grid"] = True
sns.set_context("notebook")



## 📥 Load Hipparcos Data

Place your Hipparcos CSV next to this notebook and set the filename below.  
Typical useful columns (names differ by source):
- **Parallax**: `Plx`, `parallax`, `parallax_mas` (milliarcseconds)
- **Apparent V Magnitude**: `Vmag`, `phot_g_mean_mag` (Gaia), etc.
- **Color index**: `B-V`, `B_V`, `bp_rp` (Gaia alternative)

If your file has different column names, fill in the mapping in the next cell.


In [ ]:

# 🔧 EDIT THIS: your local CSV path
DATA_PATH = "hipparcos_sample.csv"  # e.g., 'hipparcos.csv' or a subset CSV

df_raw = pd.read_csv(DATA_PATH)
print("Shape:", df_raw.shape)
df_raw.head()


In [ ]:

# Try to auto-detect common column names, then let you override if needed.

def pick_first(df, candidates):
    for c in candidates:
        if c in df.columns:
            return c
    return None

col_parallax = pick_first(df_raw, ["Plx", "parallax", "parallax_mas", "plx"])
col_vmag     = pick_first(df_raw, ["Vmag", "VmagH", "Vmagnitude", "vmag", "VmagHip"])
col_bv       = pick_first(df_raw, ["B-V", "B_V", "BV", "bv"])

print("Detected columns:")
print("  parallax:", col_parallax)
print("  Vmag    :", col_vmag)
print("  B-V     :", col_bv)

# If any are None, set them manually here, e.g.:
# col_parallax = "Plx"
# col_vmag = "Vmag"
# col_bv = "B-V"



## 🧮 Feature Engineering

We compute:
- **Distance** in parsecs: `distance_pc = 1000 / parallax_mas`
- **Absolute magnitude (Mv)** from apparent magnitude and distance:
  \[ M = m - 5 \log_{10}(d/10) \]
- Keep a clean **color index (B–V)**

We also filter physically implausible or low-quality rows.


In [ ]:

df = df_raw.copy()

# Basic sanity checks
missing_cols = [name for name in [col_parallax, col_vmag] if name is None]
if missing_cols:
    raise ValueError("Missing required column mapping. Please set col_parallax and col_vmag.")

# Rename working columns to standard names
df = df.rename(columns={
    col_parallax: "parallax_mas",
    col_vmag: "Vmag",
    **({col_bv: "B_V"} if col_bv else {})
})

# Clean parallax (must be > 0 to compute distance); cap extreme values
df = df[pd.to_numeric(df["parallax_mas"], errors="coerce") > 0].copy()
df["parallax_mas"] = df["parallax_mas"].astype(float)

# Distance in parsecs
df["distance_pc"] = 1000.0 / df["parallax_mas"]

# Absolute magnitude (Mv)
df["Mv"] = df["Vmag"] - 5 * (np.log10(df["distance_pc"]) - 1)

# If B–V not present, keep it as NaN (we can still cluster on Mv + distance or add other bands)
if "B_V" in df.columns:
    df["B_V"] = pd.to_numeric(df["B_V"], errors="coerce")

# Quality & plausibility filters (you can relax/tighten as needed)
df = df[(df["distance_pc"] > 0) & (df["distance_pc"] < 5000)]  # within 5 kpc to avoid extremes
df = df[(df["Mv"] > -10) & (df["Mv"] < 20)]                    # plausible absolute magnitudes

# Drop rows missing key features
key_feats = ["Mv", "distance_pc"]
if "B_V" in df.columns:
    key_feats.append("B_V")
df = df.dropna(subset=key_feats).copy()

print("After cleaning:", df.shape)
df[key_feats].describe()



## 📈 Hertzsprung–Russell (HR) Diagram

Classic astronomy plot: **x = B–V (color)**, **y = Mv (absolute magnitude)** (note: y-axis inverted).
If B–V is missing, we’ll plot Mv vs distance as a sanity check.


In [ ]:

has_bv = "B_V" in df.columns and df["B_V"].notna().any()

if has_bv:
    fig, ax = plt.subplots()
    ax.scatter(df["B_V"], df["Mv"], s=5, alpha=0.5)
    ax.set_xlabel("B - V (color index)")
    ax.set_ylabel("Absolute Magnitude (Mv)")
    ax.invert_yaxis()  # brighter at the top
    ax.set_title("HR Diagram (Hipparcos)")
    plt.show()
else:
    fig, ax = plt.subplots()
    ax.scatter(df["distance_pc"], df["Mv"], s=5, alpha=0.5)
    ax.set_xlabel("Distance (pc)")
    ax.set_ylabel("Absolute Magnitude (Mv)")
    ax.invert_yaxis()
    ax.set_title("Mv vs Distance (B–V not available)")
    plt.show()



## 🧱 Prepare Features & Scale

We’ll use the most informative physical features available:
- If B–V exists: **[B–V, Mv, log10(distance)]**
- Else: **[Mv, log10(distance)]**


In [ ]:

features = ["Mv", "distance_pc"]
if has_bv:
    features = ["B_V", "Mv", "distance_pc"]

X = df[features].copy()
# Log-scale distance helps
X["log10_distance"] = np.log10(X["distance_pc"])
X = X.drop(columns=["distance_pc"])

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("Feature matrix shape:", X_scaled.shape)
print("Features used:", list(X.columns))



## 🔻 PCA (for visualization only)


In [ ]:

pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X_scaled)
print("Explained variance ratio:", pca.explained_variance_ratio_)



## 🔧 KMeans: Elbow & Silhouette

We’ll try cluster counts from 2 to 8 and inspect the elbow and silhouette scores.


In [ ]:

inertias = []
sils = []
ks = range(2, 9)

for k in ks:
    km = KMeans(n_clusters=k, n_init="auto", random_state=42)
    labels = km.fit_predict(X_scaled)
    inertias.append(km.inertia_)
    sils.append(silhouette_score(X_scaled, labels))

fig, ax = plt.subplots()
ax.plot(list(ks), inertias, 'o-')
ax.set_xlabel("k (clusters)")
ax.set_ylabel("Inertia")
ax.set_title("Elbow Plot")
plt.show()

fig, ax = plt.subplots()
ax.plot(list(ks), sils, 'o-')
ax.set_xlabel("k (clusters)")
ax.set_ylabel("Silhouette Score")
ax.set_title("Silhouette vs k")
plt.show()



## ✅ Fit Final KMeans (choose k)

Set your chosen `k_final` below after inspecting the elbow/silhouette plots (often 3–5).


In [ ]:

k_final = 3  # 🔧 change if needed
km = KMeans(n_clusters=k_final, n_init="auto", random_state=42)
df["cluster_km"] = km.fit_predict(X_scaled)

print(df["cluster_km"].value_counts().sort_index())
df.groupby("cluster_km")[["Mv"] + (["B_V"] if has_bv else [])].describe()


In [ ]:

# HR diagram colored by clusters (if B–V available)
if has_bv:
    fig, ax = plt.subplots()
    sc = ax.scatter(df["B_V"], df["Mv"], c=df["cluster_km"], s=6, alpha=0.7, cmap="viridis")
    ax.set_xlabel("B - V (color index)")
    ax.set_ylabel("Absolute Magnitude (Mv)")
    ax.invert_yaxis()
    ax.set_title("HR Diagram colored by KMeans clusters")
    plt.colorbar(sc, ax=ax, label="cluster")
    plt.show()

# PCA scatter
fig, ax = plt.subplots()
sc = ax.scatter(X_pca[:,0], X_pca[:,1], c=df["cluster_km"], s=6, alpha=0.7, cmap="viridis")
ax.set_xlabel("PCA 1")
ax.set_ylabel("PCA 2")
ax.set_title("PCA projection colored by KMeans clusters")
plt.colorbar(sc, ax=ax, label="cluster")
plt.show()



## 🌀 Optional: DBSCAN (density-based)

Try DBSCAN if you suspect non-spherical clusters.


In [ ]:

# You may need to tune eps & min_samples. Start small, increase gradually.
eps = 0.6
min_samples = 15

db = DBSCAN(eps=eps, min_samples=min_samples)
labels_db = db.fit_predict(X_scaled)
df["cluster_db"] = labels_db  # -1 = noise

print("DBSCAN label counts:")
print(pd.Series(labels_db).value_counts().sort_index())

# Visualize PCA with DBSCAN labels
fig, ax = plt.subplots()
sc = ax.scatter(X_pca[:,0], X_pca[:,1], c=labels_db, s=6, alpha=0.7, cmap="tab10")
ax.set_xlabel("PCA 1")
ax.set_ylabel("PCA 2")
ax.set_title(f"PCA projection colored by DBSCAN (eps={eps}, min_samples={min_samples})")
plt.colorbar(sc, ax=ax, label="cluster")
plt.show()



## 💾 Save Outputs


In [ ]:

OUT_CSV = "hipparcos_clustered.csv"
df.to_csv(OUT_CSV, index=False)
print(f"Saved clustered dataset to {OUT_CSV}")



## 🧭 Interpreting Clusters (Quick Guide)

- **Lower Mv (top of HR diagram) = intrinsically brighter** stars.
- **Higher B–V = redder/cooler**; **Lower B–V = bluer/hotter**.
- Expect clusters roughly corresponding to:
  - **Main sequence** (diagonal band)
  - **Red giants** (upper-right)
  - **White dwarfs** (lower-left; low luminosity but blue)

Use `groupby("cluster_km").mean()` and compare typical B–V / Mv ranges to label your clusters in a table.
